# 🧹 LIMPIEZA DE DATOS - MIMIC: Detección de anomalías
Autor: Isaac Uribe
Fecha: 2025-11-09
#Descripción: Carga, inspección y preparación inicial del dataset

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
sns.set(style="whitegrid")
print("Librerías cargadas correctamente.")

Librerías cargadas correctamente.


In [118]:
ruta_datos = r"C:\Users\User\Documents\Proyectos_Programacion\MIMIC-Deteccion-anomalias\datos\df_merged_acotado.csv"

# Leer el archivo CSV
df_mimic = pd.read_csv(ruta_datos)

print("✅ Dataset cargado correctamente.")
print(f"Dimensiones: {df_mimic.shape}")
print("\nVista rápida de las primeras filas:")
display(df_mimic.head())

✅ Dataset cargado correctamente.
Dimensiones: (100000, 16)

Vista rápida de las primeras filas:


,SUBJECT_ID,HADM_ID,CHARTTIME,HEART_RATE,HEART_RATE_UOM,TEMPERATURE,TEMPERATURE_UOM,OXYGEN_SAT,OXYGEN_SAT_UOM,DIASTOLIC_BP,BLOOD_DIASTOLIC_UOM,SYSTOLIC_BP,BLOOD_SYSTOLIC_UOM,GLUCOSE,GLUCOSE_UOM,PATIENT_STATE
0,2,163353,2138-07-17 20:20:00,148.0,bpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taquicardia
1,2,163353,2138-07-17 20:30:00,131.0,bpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taquicardia
2,2,163353,2138-07-17 21:00:00,144.0,bpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taquicardia
3,2,163353,2138-07-17 22:00:00,140.0,bpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taquicardia
4,3,145834,2101-10-20 18:45:00,95.0,BPM,NaN,NaN,98.0,%,NaN,NaN,NaN,NaN,NaN,NaN,Normal


In [119]:
biomarcadores = [
    "HEART_RATE",
    "TEMPERATURE",
    "GLUCOSE",
    "OXYGEN_SAT",
    "SYSTOLIC_BP",
    "DIASTOLIC_BP"
]

columna_objetivo = "PATIENT_STATE"

 Aquí se implementarán pasos como:
 - Eliminación o imputación valores
 - Detección de outliers
 - Normalización / estandarización de variables

In [120]:
# Asegurar que CHARTTIME esté en formato datetime
df_signos_filled = df_mimic.copy()
df_signos_filled['CHARTTIME'] = pd.to_datetime(df_signos_filled['CHARTTIME'])

# Redondear CHARTTIME a la hora (para evitar diferencias por minutos)
df_signos_filled['CHARTTIME'] = df_signos_filled['CHARTTIME'].dt.floor('H')

# Ordenar por paciente, ingreso y tiempo
df_signos_filled = df_signos_filled.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME'])

# Agrupar y rellenar hacia adelante y atrás para evitar NaN dentro de misma hora
df_signos_filled = df_signos_filled.groupby(['SUBJECT_ID', 'HADM_ID', 'CHARTTIME']).first().reset_index()
df_signos_filled = df_signos_filled.groupby(['SUBJECT_ID', 'HADM_ID']).apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
df_signos_filled

C:\Users\User\AppData\Local\Temp\ipykernel_35084\2496320637.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_signos_filled = df_signos_filled.groupby(['SUBJECT_ID', 'HADM_ID']).apply(lambda group: group.ffill().bfill()).reset_index(drop=True)


,SUBJECT_ID,HADM_ID,CHARTTIME,HEART_RATE,HEART_RATE_UOM,TEMPERATURE,TEMPERATURE_UOM,OXYGEN_SAT,OXYGEN_SAT_UOM,DIASTOLIC_BP,BLOOD_DIASTOLIC_UOM,SYSTOLIC_BP,BLOOD_SYSTOLIC_UOM,GLUCOSE,GLUCOSE_UOM,PATIENT_STATE
0,2,163353,2138-07-17 20:00:00,148.0,bpm,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,Taquicardia
1,2,163353,2138-07-17 21:00:00,144.0,bpm,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,Taquicardia
2,2,163353,2138-07-17 22:00:00,140.0,bpm,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,Taquicardia
3,3,145834,2101-10-20 18:00:00,95.0,BPM,36.200001,Deg. C,98.0,%,0.0,mmHg,0.0,mmHg,140.0,None,Normal
4,3,145834,2101-10-20 19:00:00,151.0,BPM,36.200001,Deg. C,82.0,%,0.0,mmHg,0.0,mmHg,140.0,None,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75718,507,136251,2148-09-19 22:00:00,72.0,BPM,38.400002,Deg. C,98.0,%,63.0,mmHg,108.0,mmHg,137.0,None,Normal
75719,507,136251,2148-09-19 23:00:00,73.0,BPM,38.500000,Deg. C,97.0,%,52.0,mmHg,120.0,mmHg,137.0,None,Normal
75720,507,136251,2148-09-20 00:00:00,75.0,BPM,38.700001,Deg. C,98.0,%,58.0,mmHg,130.0,mmHg,114.0,None,Normal
75721,507,136251,2148-09-20 01:00:00,74.0,BPM,38.099998,Deg. C,98.0,%,51.0,mmHg,110.0,mmHg,113.0,None,Normal


En esta etapa se definen los umbrales mínimos aceptables para algunas variables fisiológicas, con el objetivo de eliminar registros que, por su valor, representarían condiciones incompatibles con la vida o errores de medición evidentes.

Temperatura corporal (TEMPERATURA):
De acuerdo con fuentes médicas ([ver Wikipedia - Hypothermia](https://en.wikipedia.org/wiki/Hypothermia?utm_source=chatgpt.com)), se considera que el cuerpo humano entra en un estado de hipotermia cuando la temperatura central desciende por debajo de los 35 °C. Temperaturas menores a este valor comprometen de manera crítica las funciones vitales, y valores inferiores a 20 °C serían prácticamente incompatibles con la vida.
Por este motivo, se eliminarán los registros con valores de TEMPERATURA menores a 20 °C, asumiendo que representan errores instrumentales o mediciones atípicas no fisiológicas.

Saturación de oxígeno (OXYGEN_SATURATION):
Según la literatura médica ([ver Medical News Today - Normal oxygen levels](https://www.medicalnewstoday.com/articles/321044?utm_source=chatgpt.com)), una saturación de oxígeno por debajo de 90 % se considera un signo de hipoxemia, lo que indica niveles insuficientes de oxígeno en la sangre y, en casos prolongados, puede ser potencialmente letal.
Por lo tanto, se establece un umbral mínimo de 90 % para esta variable, eliminando cualquier registro con valores inferiores a dicho límite.

En conjunto, estos criterios permiten mantener únicamente observaciones que representen condiciones fisiológicas realistas, evitando que datos erróneos o no humanos afecten las siguientes etapas del análisis y modelado.

In [121]:
# 1. Umbrales:
temp_umbral = 20.0   # °C
spo2_umbral = 50.0   # % (porcentaje)

# 2. Filtrar/Eliminar:
df_signos_filled = df_signos_filled.copy()

# Temperatura menor al umbral
mask_temp = df_signos_filled["TEMPERATURE"] < temp_umbral
print(f"Valores de TEMPERATURA < {temp_umbral} °C que serán eliminados: {mask_temp.sum()}")

# Saturación de oxígeno menor al umbral
mask_spo2 = df_signos_filled["OXYGEN_SAT"] < spo2_umbral
print(f"Valores de OXYGEN_SATURATION < {spo2_umbral}% que serán eliminados: {mask_spo2.sum()}")

# Aplicar la eliminación
df_signos_filled = df_signos_filled.loc[~(mask_temp | mask_spo2)].reset_index(drop=True)

print("Dimensiones tras limpieza:", df_signos_filled.shape)

Valores de TEMPERATURA < 20.0 °C que serán eliminados: 2722
Valores de OXYGEN_SATURATION < 50.0% que serán eliminados: 80
Dimensiones tras limpieza: (72922, 16)


# 🧩 Imputacion de datos faltantes 

Luego de realizar la limpieza inicial de datos atípicos y la conversión de tiempos a formato horario, persisten algunos valores faltantes en los biomarcadores principales.
Para evitar perder información valiosa y mantener la continuidad temporal de las mediciones por paciente, se aplicará una estrategia de imputación bidireccional (forward y backward fill).

In [122]:
# Aplicar imputación solo en los biomarcadores, por paciente y hospitalización
for biom in biomarcadores:
    df_signos_filled[biom] = (
        df_signos_filled
        .groupby(['SUBJECT_ID', 'HADM_ID'])[biom]
        .transform(lambda g: g.ffill().bfill())
    )

for biom in biomarcadores:
    df_signos_filled[biom] = df_signos_filled.groupby(
        ['SUBJECT_ID', 'HADM_ID']
    )[biom].transform(lambda x: x.fillna(x.median()))

# --- Verificar si aún hay valores faltantes ---
faltantes_post = df_signos_filled[biomarcadores].isna().sum()
print("Valores faltantes después de imputación (ffill + bfill por hora):")
print(faltantes_post)

Valores faltantes después de imputación (ffill + bfill por hora):
HEART_RATE         67
TEMPERATURE     55790
GLUCOSE         21082
OXYGEN_SAT      20934
SYSTOLIC_BP     31824
DIASTOLIC_BP    31824
dtype: int64


## 🧩 Imputación de valores faltantes con KNN

Dado que los valores faltantes en el dataset no son completamente aleatorios, sino que dependen del contexto clínico (frecuencia de monitoreo, estado del paciente o tipo de hospitalización), se concluye que el mecanismo de ausencia corresponde a un MAR (Missing At Random).

Por esta razón, no se eliminan las variables con alta proporción de valores faltantes, como TEMPERATURE, ya que presentan relevancia fisiológica en la interpretación de los signos vitales (por ejemplo, el aumento de temperatura suele correlacionar con el incremento del ritmo cardíaco).

En lugar de ello, se implementa una imputación mediante el algoritmo KNN (K-Nearest Neighbors Imputer), que estima los valores faltantes a partir de pacientes con características fisiológicas similares.
Este método tiene la ventaja de preservar las relaciones multivariadas entre los biomarcadores, reduciendo el sesgo introducido por imputaciones simples como la media o la mediana.

In [123]:
from sklearn.impute import KNNImputer

# Seleccionar solo biomarcadores numéricos
X = df_signos_filled[biomarcadores]

# Crear e inicializar el imputador
imputer = KNNImputer(n_neighbors=5, weights='distance')

# Aplicar imputación (KNN busca pacientes similares)
X_imputed = imputer.fit_transform(X)

# Reemplazar en el dataframe original
df_signos_filled = df_signos_filled.copy()
df_signos_filled[biomarcadores] = X_imputed

# Verificar reducción de NaN
faltantes_post_knn = df_signos_filled[biomarcadores].isna().sum()
print("Valores faltantes después de KNN:")
print(faltantes_post_knn)


Valores faltantes después de KNN:
HEART_RATE      0
TEMPERATURE     0
GLUCOSE         0
OXYGEN_SAT      0
SYSTOLIC_BP     0
DIASTOLIC_BP    0
dtype: int64


### 🧩 Casillas opcionales: 

Si después aún quedan valores faltantes, usamos la mediana global del biomarcador

In [30]:
for biom in biomarcadores:
    df_signos_filled[biom] = df_signos_filled[biom].fillna(df_signos_filled[biom].median())


In [31]:
faltantes_pct = df_signos_filled[biomarcadores].isna().mean() * 100
print("Porcentaje de valores faltantes por biomarcador (%):")
print(faltantes_pct.sort_values(ascending=False))


Porcentaje de valores faltantes por biomarcador (%):
HEART_RATE      0.0
TEMPERATURE     0.0
GLUCOSE         0.0
OXYGEN_SAT      0.0
SYSTOLIC_BP     0.0
DIASTOLIC_BP    0.0
dtype: float64


In [77]:
df_signos_filled["GLUCOSE_UOM"].unique()

array([None, 'mg/dL'], dtype=object)

In [124]:
# ============================================================
# ⚙️ Estandarización total de las unidades de medida (UOM)
# ============================================================

# Identificar columnas con sufijo "_UOM"
uom_columns = [col for col in df_signos_filled.columns if col.endswith("_UOM")]
print("Columnas con unidades de medida encontradas:", uom_columns)

# Reemplazar todos los valores por la unidad estándar correspondiente
for col in uom_columns:
    col_lower = col.lower()
    if "temperature" in col_lower:
        df_signos_filled[col] = "°C"
    elif "glucose" in col_lower:
        df_signos_filled[col] = "mg/dL"
    elif "pressure" in col_lower:
        df_signos_filled[col] = "mmHg"
    elif "oxygen" in col_lower or "spo2" in col_lower:
        df_signos_filled[col] = "%"
    elif "heart" in col_lower or "pulse" in col_lower:
        df_signos_filled[col] = "bpm"
    else:
        # Si no se identifica claramente el tipo, se marca como 'unitless'
        df_signos_filled[col] = "unitless"

# Verificación rápida
for col in uom_columns:
    print(f"{col}: {df_signos_filled[col].unique()}")


Columnas con unidades de medida encontradas: ['HEART_RATE_UOM', 'TEMPERATURE_UOM', 'OXYGEN_SAT_UOM', 'BLOOD_DIASTOLIC_UOM', 'BLOOD_SYSTOLIC_UOM', 'GLUCOSE_UOM']
HEART_RATE_UOM: ['bpm']
TEMPERATURE_UOM: ['°C']
OXYGEN_SAT_UOM: ['%']
BLOOD_DIASTOLIC_UOM: ['unitless']
BLOOD_SYSTOLIC_UOM: ['unitless']
GLUCOSE_UOM: ['mg/dL']


In [125]:
df_signos_filled

,SUBJECT_ID,HADM_ID,CHARTTIME,HEART_RATE,HEART_RATE_UOM,TEMPERATURE,TEMPERATURE_UOM,OXYGEN_SAT,OXYGEN_SAT_UOM,DIASTOLIC_BP,BLOOD_DIASTOLIC_UOM,SYSTOLIC_BP,BLOOD_SYSTOLIC_UOM,GLUCOSE,GLUCOSE_UOM,PATIENT_STATE
0,2,163353,2138-07-17 20:00:00,148.0,bpm,38.500000,°C,97.2,%,61.00,unitless,115.60,unitless,180.2,mg/dL,Taquicardia
1,2,163353,2138-07-17 21:00:00,144.0,bpm,38.200001,°C,96.6,%,58.25,unitless,101.75,unitless,115.8,mg/dL,Taquicardia
2,2,163353,2138-07-17 22:00:00,140.0,bpm,37.700001,°C,93.6,%,68.20,unitless,117.80,unitless,110.2,mg/dL,Taquicardia
3,3,145834,2101-10-20 18:00:00,95.0,bpm,36.200001,°C,98.0,%,0.00,unitless,0.00,unitless,140.0,mg/dL,Normal
4,3,145834,2101-10-20 19:00:00,151.0,bpm,36.200001,°C,82.0,%,0.00,unitless,0.00,unitless,140.0,mg/dL,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72917,507,136251,2148-09-19 22:00:00,72.0,bpm,38.400002,°C,98.0,%,63.00,unitless,108.00,unitless,137.0,mg/dL,Normal
72918,507,136251,2148-09-19 23:00:00,73.0,bpm,38.500000,°C,97.0,%,52.00,unitless,120.00,unitless,137.0,mg/dL,Normal
72919,507,136251,2148-09-20 00:00:00,75.0,bpm,38.700001,°C,98.0,%,58.00,unitless,130.00,unitless,114.0,mg/dL,Normal
72920,507,136251,2148-09-20 01:00:00,74.0,bpm,38.099998,°C,98.0,%,51.00,unitless,110.00,unitless,113.0,mg/dL,Normal


In [126]:
# Seleccionar las columnas de interés (ajusta los nombres exactos si difieren)
cols_interes = ["GLUCOSE", "TEMPERATURE", "OXYGEN_SAT"]

# Mostrar los valores mínimos y máximos de cada una
for col in cols_interes:
    if col in df_signos_filled.columns:
        valor_min = df_signos_filled[col].min()
        valor_max = df_signos_filled[col].max()
        print(f"{col}:")
        print(f"   🔹 Mínimo: {valor_min}")
        print(f"   🔹 Máximo: {valor_max}\n")
    else:
        print(f"⚠️ La columna {col} no existe en el DataFrame.\n")

GLUCOSE:
   🔹 Mínimo: 22.0
   🔹 Máximo: 1268.0

TEMPERATURE:
   🔹 Mínimo: 32.599998474121094
   🔹 Máximo: 39.700000762939446

OXYGEN_SAT:
   🔹 Mínimo: 52.0
   🔹 Máximo: 100.0



En glucosa se puden observar valores anómalos muy altos que probablemente se deban a errores de registro (por ejemplo, 1000 mg/dL en lugar de 100 mg/dL). Se aplicará una corrección dividiendo por 10 aquellos valores que superen un umbral razonable (por ejemplo, 1000 mg/dL).
Esto se comprobo despues de que los valores se concluyeran estan en un nico formato que es el internacional con el que se toman las medidas para cada signo vital, y esto al comprobar que incluso la glucosa en la unidad de medida estado unidense no rosaria ni de cerca estos valores.

In [127]:
# Aplicar la corrección a valores anómalos de glucosa
df_signos_filled.loc[df_signos_filled['GLUCOSE'] > 1000, 'GLUCOSE'] = df_signos_filled.loc[df_signos_filled['GLUCOSE'] > 1000, 'GLUCOSE'] / 10


| Biomarcador                                          | Rango “normal” aproximado                                          | Criterio para considerar atípico                       | Fuente                 |
| ---------------------------------------------------- | ------------------------------------------------------------------ | ------------------------------------------------------ | ---------------------- |
| **Ritmo cardíaco (HEART_RATE)**                      | 60 – 100 bpm (adulto en reposo) ([Mayo Clinic][1])                 | < 60 bpm (bradicardia) o > 100 bpm (taquicardia)       | ([Mayo Clinic][1])     |
| **Saturación de oxígeno (OXYGEN_SATURATION / SpO₂)** | 95 – 100 % en adultos sanos ([Verywell Health][2])                 | < 90 % → hipoxemia grave                               | ([Verywell Health][2]) |
| **Temperatura corporal (TEMPERATURE)**               | Aproximadamente 36.1 – 37.2 °C (97–99 °F) ([MedlinePlus][3])       | < 35 °C (hipotermia) o > 38-39 °C (fiebre/hipertermia) | ([MedlinePlus][3])     |
| **Glucosa (GLUCOSE)**                                | 70 – 99 mg/dL (3.9 – 5.5 mmol/L) en ayunas ([Cleveland Clinic][4]) | < 70 mg/dL (~2.8 mmol/L) o > 126 mg/dL (~7 mmol/L)     | ([Mayo Clinic][5])     |
| **Presión sistólica (SYSTOLIC_BP)**                  | Menos de 120 mm Hg ([www.heart.org][6])                            | ≥ 130 mm Hg (o ≥140 en otros criterios)                | ([NHLBI, NIH][7])      |
| **Presión diastólica (DIASTOLIC_BP)**                | Menos de 80 mm Hg ([www.heart.org][6])                             | ≥ 80-90 mm Hg                                          | ([WebMD][8])           |

[1]: https://www.mayoclinic.org/healthy-lifestyle/fitness/expert-answers/heart-rate/faq-20057979?utm_source=chatgpt.com "What's a normal resting heart rate?"
[2]: https://www.verywellhealth.com/normal-adult-vital-signs-914750?utm_source=chatgpt.com "Normal Vital Signs for Adults"
[3]: https://medlineplus.gov/ency/article/001982.htm?utm_source=chatgpt.com "Body temperature norms: MedlinePlus Medical Encyclopedia"
[4]: https://my.clevelandclinic.org/health/diagnostics/12363-blood-glucose-test?utm_source=chatgpt.com "Blood Glucose (Sugar) Test: Levels & What They Mean"
[5]: https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-20371451?utm_source=chatgpt.com "Diabetes - Diagnosis and treatment - Mayo Clinic"
[6]: https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings?utm_source=chatgpt.com "Understanding Blood Pressure Readings"
[7]: https://www.nhlbi.nih.gov/health/high-blood-pressure?utm_source=chatgpt.com "What Is High Blood Pressure? - Nhlbi.nih.gov"
[8]: https://www.webmd.com/hypertension-high-blood-pressure/diastolic-and-systolic-blood-pressure-know-your-numbers?utm_source=chatgpt.com "Diastole vs. Systole: Know Your Blood Pressure Numbers - WebMD"


Como se puede observar en el análisis del tratamiento de variables, este dataset presenta ciertas limitaciones derivadas del desconocimiento sobre las condiciones bajo las cuales fueron tomadas algunas mediciones, así como la presencia de valores que resultan difíciles de explicar con el conocimiento clínico actual. En particular, el manejo de datos atípicos en variables como glucosa y saturación de oxígeno requiere precaución, para evitar introducir sesgos significativos o comprometer la integridad del conjunto de datos. Teniendo en cuenta lo anterior, se implementaron estrategias de limpieza e imputación orientadas a conservar la mayor cantidad de información posible sin alterar la coherencia clínica de las mediciones.

# Conclusiones de la limpieza

A lo largo del proceso de limpieza del dataset se abordaron múltiples etapas orientadas a mejorar la calidad y coherencia de los datos biomédicos. En primer lugar, se realizó la corrección de unidades de medida, transformando los valores de temperatura (de Fahrenheit a Celsius) para mantener consistencia en el análisis. Posteriormente, se implementaron estrategias de detección y tratamiento de valores atípicos, utilizando métodos estadísticos como el Z-score y el IQR, complementados con una interpretación clínica basada en rangos fisiológicamente posibles y teniendo en cuenta las 3 clases que son nuestro interes de estudio en este data set con deteccion de atipicos.

Asimismo, se aplicaron técnicas de imputación de valores faltantes, priorizando la coherencia temporal mediante el rellenado hacia adelante y hacia atrás por paciente y hospitalización, y considerando posteriormente el uso de enfoques más avanzados como la imputación por KNN, dada la naturaleza parcialmente condicionada de los datos faltantes (MAR).

Adicionalmente, se realizaron ajustes en mediciones anómalas de glucosa (corrigiendo posibles errores de escala), y se definieron umbrales clínicos de exclusión para temperatura y saturación de oxígeno, basados en literatura médica. Estas acciones permitieron reducir la cantidad de valores inconsistentes y conservar la validez fisiológica de las observaciones.

En conjunto, el proceso de limpieza logró un equilibrio entre la preservación de la información y la integridad clínica de las variables, dejando un dataset más confiable y listo para análisis posteriores de detección de anomalías o modelado predictivo.

In [129]:
ruta_salida = r"C:\Users\User\Documents\Proyectos_Programacion\MIMIC-Deteccion-anomalias\datos\df_mimic_limpio.csv"
df_signos_filled.to_csv(ruta_salida, index=False)
print(f"✅ Dataset limpio guardado correctamente en:\n{ruta_salida}")

✅ Dataset limpio guardado correctamente en:
C:\Users\User\Documents\Proyectos_Programacion\MIMIC-Deteccion-anomalias\datos\df_mimic_limpio.csv
